In [1]:
# in reference to:
# https://towardsdatascience.com/scraping-reddit-with-praw-76efc1d1e1d9
# https://praw.readthedocs.io/en/latest/code_overview/praw_models.html
# https://www.reddit.com/wiki/search

In [1]:
from os.path import isfile
import praw
import pandas as pd
from time import sleep

# Get credentials from DEFAULT instance in praw.ini
# Use your own tokens
reddit = praw.Reddit(
    client_id="tpkjzcHMAynTvw",
    client_secret="WdtCdrgnWMfZphmCgkl_fVFH-uA",
    user_agent="scraper",
    username="Whatchamacalit1",
    password="Tango9802!"
)


class SubredditScraper:

    def __init__(self, query, sort='new', lim=900, mode='w'):
        self.query = query
        self.sort = sort
        self.lim = lim
        self.mode = mode

        print(
            f'SubredditScraper instance created with values '
            f'query = {query}, sort = {sort}, lim = {lim}, mode = {mode}')

    def set_sort(self):
        if self.sort == 'new':
            return self.sort, reddit.subreddit('all').search(self.query, sort='new', time_filter='month')
        elif self.sort == 'top':
            return self.sort, reddit.subreddit('all').search(self.query, sort='top', time_filter='month')
        elif self.sort == 'hot':
            return self.sort, reddit.subreddit('all').search(self.query, sort='hot', time_filter='month')
        else:
            self.sort = 'relevance'
            print('Sort method was not recognized, defaulting to relevance.')
            return self.sort, reddit.subreddit('all').search(self.query, time_filter='month')

    def get_posts(self, printer=False):

        sub_dict = {
            'game_title': [], 'selftext': [], 'title': [], 'id': [], 'sorted_by': [],
            'num_comments': [], 'score': [], 'ups': [], 'downs': []}
        csv = f'game_titles_1.csv'

        # Attempt to specify a sorting method.
        sort, subreddit = self.set_sort()

        # Set csv_loaded to True if csv exists since you can't evaluate the
        # truth value of a DataFrame.
        df, csv_loaded = (pd.read_csv(csv), 1) if isfile(csv) else ('', 0)

        print(f'csv = {csv}')
        print(f'After set_sort(), sort = {sort} and query = {self.query}')
        print(f'csv_loaded = {csv_loaded}')

        print(f'Collecting information from r/all for search term "{self.query}".')
        for post in subreddit:
            # Save any unique, non-stickied posts with descriptions to sub_dict.
            sub_dict['game_title'].append(self.query)
            sub_dict['selftext'].append(post.selftext)
            sub_dict['title'].append(post.title)
            sub_dict['id'].append(post.id)
            sub_dict['sorted_by'].append(sort)
            sub_dict['num_comments'].append(post.num_comments)
            sub_dict['score'].append(post.score)
            sub_dict['ups'].append(post.ups)
            sub_dict['downs'].append(post.downs)
            sleep(0.1)

        sub_dict_final = {
            'game_title': [self.query],  
            'score': [sum(sub_dict['score'])], 
            'ups': [sum(sub_dict['ups'])], 
            'downs': [sum(sub_dict['downs'])]}
        
        # print(sub_dict)
        new_df = pd.DataFrame(sub_dict_final)
        
        # decide whether or no to print
        if printer is True:
            print(
                f'{len(sub_dict)} posts were collected but they were not '
                f'added to {csv} because mode was set to "{self.mode}"')
            return

        # Add new_df to df if df exists then save it to a csv.
        if 'DataFrame' in str(type(df)) and self.mode == 'w':
            pd.concat([df, new_df], axis=0, sort=0).to_csv(csv, index=False)
            print(
                f'{len(new_df)} new posts collected and added to {csv}')
        elif self.mode == 'w':
            new_df.to_csv(csv, index=False)
            print(f'{len(new_df)} posts collected and saved to {csv}')
        else:
            print(
                f'{len(new_df)} posts were collected but they were not '
                f'added to {csv} because mode was set to "{self.mode}"')


#if __name__ == '__main__':
    #SubredditScraper('statistics', lim=997, mode='w', sort='new').get_posts()
    #SubredditScraper('Over Watch', lim=997, sort='new').get_posts()

In [3]:
all_titles = pd.read_csv("All_Games.csv", error_bad_lines=False)

b'Skipping line 252: expected 8 fields, saw 9\nSkipping line 284: expected 8 fields, saw 10\nSkipping line 396: expected 8 fields, saw 9\nSkipping line 609: expected 8 fields, saw 9\nSkipping line 610: expected 8 fields, saw 9\nSkipping line 628: expected 8 fields, saw 9\nSkipping line 696: expected 8 fields, saw 9\nSkipping line 1561: expected 8 fields, saw 9\nSkipping line 2536: expected 8 fields, saw 9\nSkipping line 2650: expected 8 fields, saw 9\nSkipping line 2726: expected 8 fields, saw 9\nSkipping line 2773: expected 8 fields, saw 9\nSkipping line 2891: expected 8 fields, saw 11\nSkipping line 2892: expected 8 fields, saw 10\nSkipping line 2983: expected 8 fields, saw 9\nSkipping line 3005: expected 8 fields, saw 9\nSkipping line 3094: expected 8 fields, saw 10\nSkipping line 3119: expected 8 fields, saw 9\nSkipping line 3141: expected 8 fields, saw 10\nSkipping line 3151: expected 8 fields, saw 13\nSkipping line 3171: expected 8 fields, saw 17\nSkipping line 3289: expected 8 f

In [4]:
# [[0, 1283], [1283, 2566], [2566, 3849], [3849, 5132], [5132, 6417]]

for i in range(1283, 2566):
    print('Scraping: (' + str(i) + ')')
    title = all_titles.title[i]
    SubredditScraper(str(title), lim=997, sort='new').get_posts()

Scraping: (1283)
SubredditScraper instance created with values query = Slime-san: Super Slime Edition, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Slime-san: Super Slime Edition
csv_loaded = 0
1 posts collected and saved to game_titles_1.csv
Scraping: (1284)
SubredditScraper instance created with values query = Small Radios Big Televisions, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Small Radios Big Televisions
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1285)
SubredditScraper instance created with values query = SmuggleCraft, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = SmuggleCraft
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1286)
SubredditScraper instance created with values query = Snake Pass, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After se

1 new posts collected and added to game_titles_1.csv
Scraping: (1308)
SubredditScraper instance created with values query = Spelunky 2, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Spelunky 2
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1309)
SubredditScraper instance created with values query = Spider: Rite of the Shrouded Moon, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Spider: Rite of the Shrouded Moon
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1310)
SubredditScraper instance created with values query = Spider-Man, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Spider-Man
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1311)
SubredditScraper instance created with values query = Spintires: MudRunner, sort = new, lim = 997, mode = w

1 new posts collected and added to game_titles_1.csv
Scraping: (1332)
SubredditScraper instance created with values query = Stealth Inc. 2, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Stealth Inc. 2
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1333)
SubredditScraper instance created with values query = SteamWorld Dig, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = SteamWorld Dig
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1334)
SubredditScraper instance created with values query = SteamWorld Dig 2, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = SteamWorld Dig 2
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1335)
SubredditScraper instance created with values query = SteamWorld Heist, sort = new, lim = 997, mode = w
csv = game_titles_1.c

1 new posts collected and added to game_titles_1.csv
Scraping: (1356)
SubredditScraper instance created with values query = Styx: Master of Shadows, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Styx: Master of Shadows
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1357)
SubredditScraper instance created with values query = Styx: Shards of Darkness, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Styx: Shards of Darkness
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1358)
SubredditScraper instance created with values query = Sublevel Zero Redux, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Sublevel Zero Redux
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1359)
SubredditScraper instance created with values query = Subnautica, sort = new, lim

1 new posts collected and added to game_titles_1.csv
Scraping: (1380)
SubredditScraper instance created with values query = Super Robot Wars V, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Super Robot Wars V
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1381)
SubredditScraper instance created with values query = Super Rude Bear Resurrection, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Super Rude Bear Resurrection
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1382)
SubredditScraper instance created with values query = Super Stardust Ultra, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Super Stardust Ultra
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (1383)
SubredditScraper instance created with values query = Super Star Wars, sort = new

...

1 new posts collected and added to game_titles_1.csv
Scraping: (2458)
SubredditScraper instance created with values query = Red Faction: Guerrilla, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Red Faction: Guerrilla
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2459)
SubredditScraper instance created with values query = Remember Me, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Remember Me
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2460)
SubredditScraper instance created with values query = Resident Evil Chronicles HD Remaster, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Resident Evil Chronicles HD Remaster
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2461)
SubredditScraper instance created with values query = Resident Evil: Operat

1 new posts collected and added to game_titles_1.csv
Scraping: (2482)
SubredditScraper instance created with values query = Rocksmith, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Rocksmith
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2483)
SubredditScraper instance created with values query = Rogue Warrior, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Rogue Warrior
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2484)
SubredditScraper instance created with values query = Root Double: Before Crime * After Days Xtend edition, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Root Double: Before Crime * After Days Xtend edition
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2485)
SubredditScraper instance created with values query = Rugby Leagu

1 new posts collected and added to game_titles_1.csv
Scraping: (2505)
SubredditScraper instance created with values query = SBK-08: Superbike World Championship, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = SBK-08: Superbike World Championship
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2506)
SubredditScraper instance created with values query = SBK-09: Superbike World Championship, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = SBK-09: Superbike World Championship
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2507)
SubredditScraper instance created with values query = SBK 2011: Superbike World Championship, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = SBK 2011: Superbike World Championship
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: 

1 new posts collected and added to game_titles_1.csv
Scraping: (2527)
SubredditScraper instance created with values query = Shaun White Snowboarding, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Shaun White Snowboarding
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2528)
SubredditScraper instance created with values query = Shellshock 2: Blood Trails, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Shellshock 2: Blood Trails
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2529)
SubredditScraper instance created with values query = Shift 2: Unleashed, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Shift 2: Unleashed
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2530)
SubredditScraper instance created with values query = Shin Hayarigami, sort =

1 new posts collected and added to game_titles_1.csv
Scraping: (2551)
SubredditScraper instance created with values query = Singstar Made in Germany, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = Singstar Made in Germany
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2552)
SubredditScraper instance created with values query = SingStar Mallorca Party, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = SingStar Mallorca Party
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2553)
SubredditScraper instance created with values query = SingStar Mecano, sort = new, lim = 997, mode = w
csv = game_titles_1.csv
After set_sort(), sort = new and query = SingStar Mecano
csv_loaded = 1
1 new posts collected and added to game_titles_1.csv
Scraping: (2554)
SubredditScraper instance created with values query = SingStar: Morangos com Acucar, sort